<a href="https://colab.research.google.com/github/KrishnaPothula/stock_market_analysis/blob/main/Hist_Stock_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import yfinance as yf
import numpy as np

# Define the stock symbol
symbol = 'WIPRO.NS'
print(symbol)
# Fetch data from Yahoo Finance
df = yf.download(symbol)

# Create a column for quarters
df['Quarter'] = df.index.to_period('Q')

# Group by quarter and calculate the difference between the last and first day of the quarter
quarterly_diff = df.groupby('Quarter')['Close'].apply(lambda x: x.iloc[-1] - x.iloc[0])

# Count the number of positive and negative values for each quarter
quarterly_counts = quarterly_diff.groupby(quarterly_diff.index.quarter).agg(['count', lambda x: (x > 0).sum(), lambda x: (x < 0).sum()])
quarterly_counts.columns = ['Total', 'Positive', 'Negative']

# Calculate the percentage of positive values for each quarter
quarterly_counts['Positive Percentage'] = quarterly_counts['Positive'] / quarterly_counts['Total']

# Create a column for the output based on the percentage of positive values
conditions = [
    (quarterly_counts['Positive Percentage'] > 0.67),
    (quarterly_counts['Positive Percentage'] <= 0.67) & (quarterly_counts['Positive Percentage'] > 0.51),
    (quarterly_counts['Positive Percentage'] <= 0.41),
    (quarterly_counts['Positive Percentage'] > 0.41) & (quarterly_counts['Positive Percentage'] <= 0.51)
]
choices = ['Strong Buy', 'Weak Buy', 'Strong Sell', 'Weak Sell']
quarterly_counts['Output'] = np.select(conditions, choices)

# Print the final output
for index, row in quarterly_counts.iterrows():
    print(f"Q{index}: {row['Output']} ({row['Positive']}, {row['Negative']})")


WIPRO.NS
[*********************100%***********************]  1 of 1 completed
Q1: Weak Buy (15, 13)
Q2: Weak Sell (14, 14)
Q3: Weak Sell (14, 13)
Q4: Strong Buy (19, 8)
